In [ ]:
import pandas as pd
from shopifyapi import ShopifyApp
import os
from dotenv import load_dotenv
import re
import csv

In [ ]:
load_dotenv()

# Support Functions

In [ ]:
def adding_upc(df):
    try:
        upc_df = pd.read_csv('available_upc.csv')
        upc_df.fillna('', inplace=True)
        for index in range(len(df)):
            if pd.notna(df.loc[index, 'Variant SKU']):
                df.loc[index, 'Variant Barcode'] = upc_df[upc_df['Available'] == ''].iloc[0, 0].copy()
                df.loc[index, 'Google Shopping / MPN'] = upc_df[upc_df['Available'] == ''].iloc[0, 0].copy()
                index = upc_df.index[upc_df['Available'] == ''][0]
                upc_df.iloc[index, 1] = 'Used'
            else:
                df.loc[index, 'Variant Barcode'] = ''
                df.loc[index, 'Google Shopping / MPN'] = ''
        upc_df.to_csv('available_upc.csv', index=False)

        return df
    
    except IndexError:
        print('UPC are not available!')

In [ ]:
def adding_tags(title, tags):
    try:
        if pd.notna(title):
            if pd.notna(tags):
                result = tags + ', toys'
            else:
                result = 'toys'
        else:
            result = None
        
        return result
    except Exception as e:
        print(e)
            

In [ ]:
def clean_text(text):
    if pd.isna(text):
        return text
    # First replace words containing 'freddo' with 'Trendtimes'
    intermediate_text = re.sub(r'\w*freddo\w*', 'Magic Cars', text, flags=re.IGNORECASE)
    
    # Then remove 'dti direct' (case insensitive)
    cleaned_text = re.sub(r'dti\s+direct', 'Magic Cars', intermediate_text, flags=re.IGNORECASE)
    
    return cleaned_text

In [ ]:
def clean_handle(text):
    if pd.isna(text):
        return text
    # First replace words containing 'freddo' with 'Trendtimes'
    intermediate_text = re.sub(r'\w*freddo\w*', 'magic-cars', text, flags=re.IGNORECASE)
    
    # Then remove 'dti direct' (case insensitive)
    cleaned_text = re.sub(r'dti-direct', 'magic-cars', intermediate_text, flags=re.IGNORECASE)
    
    return cleaned_text

In [ ]:
def remove_video_embeds(text):
    if pd.isna(text):
        return text
    
    # pattern = r"<h2>[^<]*?video[^<]*?</h2>\s*<p>.*?<iframe.*?</iframe>\s*</p>"
    pattern = r"<h2>[^<]*?video[^<]*?</h2>\s*(<p>\s*<iframe.*?</iframe>\s*</p>|<iframe.*?</iframe>)"
    cleaned_text = re.sub(pattern, "", text, flags=re.DOTALL | re.IGNORECASE)
    
    return cleaned_text


In [ ]:
def remove_manual_embeds(text):
    if pd.isna(text):
        return text
    
    # pattern = r"<h2>[^<]*?video[^<]*?</h2>\s*<p>.*?<iframe.*?</iframe>\s*</p>"
    # manual_pattern = r"<h2[^>]*>Documents</h2>\s*<table[^>]*>.*?</table>|<table[^>]*>(?:[^<]*|<(?!table)[^>]*>)*?(?:Manual|manual)(?:[^<]*|<(?!table)[^>]*>)*?</table>"
    # cleaned_text = re.sub(manual_pattern, "", text, flags=re.DOTALL | re.IGNORECASE)

     # Create a regex pattern to match content between the start and end markers
    pattern = re.escape("<h2>Documents</h2>") + r".*?" + re.escape("</table>")
    # Remove the block using regex
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
    
    return cleaned_text

In [ ]:
def reduce_price(price):
    if pd.isna(price):
        return price
    float_price = float(price)
    if (price is None) or (float_price == 0) or (price == '0.00'):
        result = "0.00"
    else:
        result = float_price - round(float_price * 2 / 100, 2)

    return f"{result:.2f}"

In [ ]:
def modify_sku(sku, warranty, plate):
    if pd.isna(sku):
        return sku
    if warranty == 'None - $0':
        if plate == 'None - $0':
            return sku
        elif plate == 'Custom license plate - $39':
            return sku + '-P'
    elif warranty == '1 year - $89':
        if plate == 'None - $0':
            return sku + '-W'
        elif plate == 'Custom license plate - $39':
            return sku + '-WP'

In [ ]:
def modify_price(price, warranty, plate):
    if pd.isna(price):
        return price
    price = float(price)
    if warranty == 'None - $0':
        if plate == 'None - $0':
            return price
        elif plate == 'Custom license plate - $39':
            return price + 39.00
    elif warranty == '1 year - $89':
        if plate == 'None - $0':
            return price + 89.00
        elif plate == 'Custom license plate - $39':
            return price + 39.00 + 89.00

In [ ]:
def shift_option(df):
    for index in range(len(df)):
        if pd.isna(df.loc[index, 'Option1 Value']):
            df.loc[index, 'Option1 Name'] = df.loc[index, 'Option2 Name']
            df.loc[index, 'Option1 Value'] = df.loc[index, 'Option2 Value']
            df.loc[index, 'Option2 Name'] = df.loc[index, 'Option3 Name']
            df.loc[index, 'Option2 Value'] = df.loc[index, 'Option3 Value']
            df.loc[index, 'Option3 Name'] = ''
            df.loc[index, 'Option3 Value'] = ''
        else:
            pass
    
    return df    

# Read Data

## Freddotoys

In [ ]:
df_ft = pd.read_csv('freddotoys_products.csv')

## DTIDirect

In [ ]:
df_dd = pd.read_csv('dtidirect_products.csv')

# Selecting DTI Product

In [ ]:
ft_checker = df_ft[['Variant SKU']].drop_duplicates()

In [ ]:
ft_checker['is_duplicated'] = 1

In [ ]:
ft_checker.nunique()

In [ ]:
df_dd = df_dd.merge(ft_checker, how='left', on='Variant SKU')

In [ ]:
duplicated_handles = df_dd[df_dd['is_duplicated'] == 1]['Handle'].unique().tolist()

In [ ]:
df_dd.loc[df_dd['Handle'].isin(duplicated_handles), 'is_duplicated'] = 1

In [ ]:
cleaned_df_dd = df_dd[df_dd['is_duplicated'] != 1]

In [ ]:
cleaned_df_dd.drop(columns='is_duplicated', inplace=True)

# Get Collection

In [ ]:
df = pd.concat([df_ft, df_dd], ignore_index=True)

In [ ]:
df['Collection Rule'] = df['Tags'].str.split(',')

In [ ]:
parent_product_df = df[~pd.isna(df['Title'])]

In [ ]:
exploded_parent_product_df = parent_product_df.explode('Collection Rule', ignore_index=True)

In [ ]:
exploded_parent_product_df.tail()

In [ ]:
exploded_parent_product_df['Collection Rule'] = exploded_parent_product_df['Collection Rule'].str.strip()

In [ ]:
exploded_parent_product_df['Collection Name'] = exploded_parent_product_df['Collection Rule'].str.title()

In [ ]:
collection_df = exploded_parent_product_df[['Collection Name', 'Collection Rule']]

In [ ]:
collection_df.dropna(inplace=True, ignore_index=True)

In [ ]:
collection_df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
collection_df

In [ ]:
collection_df.to_csv('fredo and dti collection.csv', index=False) 

# Shopify

In [ ]:
shopify = ShopifyApp(store_name=os.getenv('STORE_NAME'), access_token=os.getenv('ACCESS_TOKEN'))
client = shopify.create_session()

In [ ]:
for index in range(len(collection_df)):
    print(collection_df.iloc[index, 0])

In [ ]:
has_next_page = True
cursor = None
results = list()
while has_next_page:
    response = shopify.get_collections(client, cursor=cursor)
    records = response['data']['collections']['nodes']
    results.extend(records)
    has_next_page = response['data']['collections']['pageInfo']['hasNextPage']
    cursor = response['data']['collections']['pageInfo']['endCursor']
results_df = pd.DataFrame.from_records(results)
results_df.to_csv('collection_list.csv', index=False)

In [ ]:
shopify_col_df = pd.read_csv('collection_list.csv')

In [ ]:
collection_df['Handle'] = collection_df.apply(lambda x: to_handle(title=x['Collection Name'], alt_title=''), axis=1)

In [ ]:
collection_df = collection_df.merge(shopify_col_df, how='left', left_on='Handle', right_on='handle')

In [ ]:
collection_df

In [ ]:
collection_df[~pd.isna(collection_df['id'])]

In [ ]:
collection_df[pd.isna(collection_df['id'])]

## Data Report

In [ ]:
products_data = products_df[pd.notna(products_df['Variant SKU'])] 

In [ ]:
products_data.loc[products_data.duplicated('Variant SKU', keep='first'), 'is_duplicated'] = 1

In [ ]:
products_data[products_data['is_duplicated'] == 1][['Handle','Variant SKU', 'Vendor']]

In [ ]:
products_data.to_csv('products_data.csv', index=False)

## Data import

In [ ]:
products_df = pd.concat([df_ft, cleaned_df_dd], ignore_index=True)

In [ ]:
products_df[pd.notna(products_df['Variant SKU'])][['Handle','Variant SKU', 'Vendor']]

# Adding Tags

In [ ]:
products_df = pd.read_csv('product_data/products_with_upc.csv')

In [ ]:
products_df['Tags'] = products_df.apply(lambda x: adding_tags(x['Title'], x['Tags']), axis=1)

In [ ]:
products_df[['Handle','Title','Tags', 'Vendor']]

# Change Vendor

In [ ]:
products_df['Vendor']

In [ ]:
products_df['Vendor'] = products_df['Vendor'].apply(lambda x: 'Htrn71' if x == 'Hobbytron' else None)

In [ ]:
products_df['Vendor']

In [ ]:
products_df.to_csv('products_with_upc_upd_vendor_tags_rev1.csv', index=False)

# Fill null option value

In [ ]:
product_df = pd.read_csv('product_data/products_with_upc_upd_vendor_tags_rev1.csv')

In [ ]:
product_df

In [ ]:
product_df.loc[(pd.notna(product_df['Option1 Name'])) & (pd.isna(product_df['Option1 Value'])), ['Option1 Value']] = 'No Battery'

In [ ]:
product_df[(pd.notna(product_df['Option3 Name'])) & (pd.isna(product_df['Option3 Value']))]

In [ ]:
product_df.info()

In [ ]:
product_df.loc[pd.notna(product_df['Title']), 'Google Shopping / Custom Label 0'] = 'Htrn71'

In [ ]:
product_df.loc[pd.notna(product_df['Title']), 'Google Shopping / Custom Label 0']

In [ ]:
product_df.to_csv('products_with_upc_upd_vendor_tags_rev2.csv', index=False)

# Remove Fredotoys and dti direct word

In [ ]:
# products_df = pd.read_csv('final_products.csv')

In [ ]:
products_df[products_df['Body (HTML)'].str.contains('freddo', case=False, na=False)]

In [ ]:
products_df['Body (HTML)'] = products_df['Body (HTML)'].apply(clean_text)

In [ ]:
products_df.iloc[34,2]

In [ ]:
products_df['Title'] = products_df['Title'].apply(clean_text)

In [ ]:
products_df['Handle'] = products_df['Handle'].apply(clean_handle)

In [ ]:
products_df.iloc[34,0]

In [ ]:
products_df[products_df['Handle'].str.contains('freddo', case=False, na=False)]

# Remove Embeded Video from body html

In [ ]:
products_df['Body (HTML)'] = products_df['Body (HTML)'].apply(remove_video_embeds)

In [ ]:
products_df['Body (HTML)'] = products_df['Body (HTML)'].apply(remove_manual_embeds)

In [ ]:
products_df.iloc[34,2]

In [ ]:
products_df[products_df['Handle'].str.contains('magic-cars', case=False, na=False)]

In [ ]:
products_df.iloc[112,1]

# Reduce Price

In [ ]:
products_df['Variant Price'] = products_df['Variant Price'].apply(reduce_price)

# Generate Inventory csv

In [ ]:
inventory_df = products_df[['Handle', 'Title', 'Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value', 'Variant SKU', 'Variant Inventory Qty']].copy()

In [ ]:
inventory_df = inventory_df[pd.notna(inventory_df['Variant SKU'])]

In [ ]:
inventory_df['L']

# Add Warranty and Custom Plate Option

In [ ]:
products_df.loc[pd.notna(products_df['Title']), 'Option2 Name'] = 'Warranty' 

In [ ]:
products_df['Option2 Value'] = products_df.apply(lambda x: ['None - $0', '1 year - $89'] if pd.notna(x['Variant SKU']) else '', axis=1)

In [ ]:
products_df.loc[pd.notna(products_df['Title']), 'Option3 Name'] = 'Custom license plate' 

In [ ]:
products_df['Option3 Value'] = products_df.apply(lambda x: ['None - $0', 'Custom license plate - $39'] if pd.notna(x['Variant SKU']) else '', axis=1)

In [ ]:
products_df = products_df.explode('Option2 Value')

In [ ]:
with open('variant_unused_columns.csv', 'r') as file:
			rows = csv.reader(file)
			variant_unused_columns = [row[0] for row in rows]

In [ ]:
products_df.loc[products_df.duplicated(['Handle', 'Option1 Value'], keep='first'), variant_unused_columns] = ''

In [ ]:
products_df = products_df.explode('Option3 Value')

In [ ]:
products_df.loc[products_df.duplicated(['Handle', 'Option2 Value'], keep='first'), variant_unused_columns] = ''

In [ ]:
products_df.loc[products_df['Handle']=='12v-magic-cars-kids-cruiser-1-seater-motorcycle', ['Handle','Title', 'Image Src', 'Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value']]

# Modify Variant SKU

In [ ]:
products_df['Variant SKU_1'] = products_df.apply(lambda x: modify_sku(x['Variant SKU'], x['Option2 Value'], x['Option3 Value']), axis=1)

In [ ]:
products_df.drop(columns='Variant SKU', inplace=True)

In [ ]:
products_df.rename({'Variant SKU_1': 'Variant SKU'}, axis=1, inplace=True)

# Modify Variant Price

In [ ]:
products_df['Variant Price_1'] = products_df.apply(lambda x: modify_price(x['Variant Price'], x['Option2 Value'], x['Option3 Value']), axis=1)

In [ ]:
products_df.drop(columns='Variant Price', inplace=True)

In [ ]:
products_df.rename({'Variant Price_1': 'Variant Price'}, axis=1, inplace=True)

# Adding UPC

In [ ]:
products_df.reset_index(drop=True, inplace=True)

In [ ]:
products_df[pd.notna(products_df['Variant SKU'])][['Handle', 'Variant SKU']]

In [ ]:
products_df = adding_upc(products_df)

In [ ]:
# Checking
products_df[products_df['Handle']=='12v-magic-cars-kids-cruiser-1-seater-motorcycle'][['Handle','Title','Variant SKU', 'Variant Price', 'Variant Barcode', 'Image Src','Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value']].to_csv('cek.csv', index=False)

In [ ]:
products_df = shift_option(products_df)

In [ ]:
products_df.to_csv('import_file/final_products.csv', index=False)